## Projekt import

In [ ]:
import types
import pandas as pd
import draf
from pathlib import Path
from h2iport import util, prep, paths
from h2iport.components import *
from h2iport.config import Config, get_dotmap_from_yml
import plotly.express as px

draf.Scenario.make_sankey_string_from_collectors = util.my_sankey_builder

cf = Config.cf

cf.consumer_data = get_dotmap_from_yml(paths.DATA_DIR / "conf2023/consumer_data_state.yml")

coords = tuple(cf.main.coords)
components = [BES, Con, Dem, DHN, EG, EHP, Elc, HDS, Pip, Lan, Main, OnPV, PV, TES, Tra, WT, PPA, REC]

## Projekt optimization

In [ ]:
cf.projections.years

In [ ]:
cf.consumer_data

In [ ]:
cf.comp.Lan

In [ ]:
cf.main.landing_settings.ships_per_day[2027]

In [ ]:
cs = draf.CaseStudy(f"h2iport", year=2019, freq="60min", coords=coords, consider_invest=True)
#cs.set_time_horizon(start="Jan-1 00:00", steps=cs.steps_per_day * 30 * 2)
sc = cs.add_REF_scen(components=components)

In [ ]:
# show total demand
sc.params.dH_Dem_TYAR.unstack(2).sum()

In [ ]:
cs.optimize(solver_params=dict(MIPGap=0.01))
# sc.export_model()

cs.save()

In [ ]:
cs.plot.sankey_interact()

In [ ]:
sc.collector_values

In [ ]:
cs.plot()

In [ ]:
cf.projections.default_year

In [ ]:
sc.vars.dQ_EHP_CAPn_Y

In [ ]:
a = sc.vars.P_PV_OC_TY.values()
b = sc.vars.P_PV_FI_TY.values()
u = 0
for i in range(0,(2*8760)):
    if u < (a[i].x + b[i].x):
        u = (a[i].x + b[i].x)
u

## If closed

### Load latest

In [ ]:
cs = draf.open_latest_casestudy("h2iport")

In [ ]:
sc = cs.scens.REF

### Load version

In [ ]:
import draf.paths as pa

In [ ]:
cs = draf.open_casestudy(pa.RESULTS_DIR / R"h2iport/XXX.p")

In [ ]:
sc = cs.scens.REF

## Plottings

### Plot Base

In [ ]:
cs.plot()

In [ ]:
cs.plot.sankey_interact()

### Electricity demand Rheinhafen

In [ ]:
year = 2030

With EHP

In [ ]:
df = pd.concat([cs.scens.REF.res.P_BES_in_TY[:,year],cs.scens.REF.res.P_EHP_TYEC[:,year,"Elc","DHN_winter"],cs.scens.REF.res.P_EHP_TYEC[:,year,"Elc","DHN_summer"],cs.scens.REF.res.P_EHP_TYEC[:,year,"amb","DHN_winter"],cs.scens.REF.res.P_EHP_TYEC[:,year,"amb","DHN_summer"],cs.scens.REF.res.P_EHP_TYEC[:,year,"rhine_water","DHN_winter"],cs.scens.REF.res.P_EHP_TYEC[:,year,"rhine_water","DHN_summer"],cs.scens.REF.res.P_Elc_TY[:,year],cs.scens.REF.res.dH_Con_out_TYRR[:,year,"CH2","NH3"]*cs.scens.REF.params.k_Con_el_RR["CH2","NH3"],cs.scens.REF.res.dH_Con_out_TYRR[:,year,"NH3","CH2"]*cs.scens.REF.params.k_Con_el_RR["NH3","CH2"]],axis=1)

Without EHP

In [ ]:
df = pd.concat([cs.scens.REF.res.P_BES_in_TY[:,year],cs.scens.REF.res.P_Elc_TY[:,year],cs.scens.REF.res.dH_Con_out_TYRR[:,year,"CH2","NH3"]*cs.scens.REF.params.k_Con_el_RR["CH2","NH3"],cs.scens.REF.res.dH_Con_out_TYRR[:,year,"NH3","CH2"]*cs.scens.REF.params.k_Con_el_RR["NH3","CH2"]],axis=1)

In [ ]:
df.plot.bar(stacked = True)

### Show costs per year

In [ ]:
res = util.get_price_per_year(self=sc, year=2027)

In [ ]:
tot = (res["C_TOT_op_sum_"] + res["C_TOT_RMI_sum_"] + res["C_TOT_invAnn_sum_"]) * 1000
print("Operation: " +str(res["C_TOT_op_sum_"]* 1000))
print("Repair:    " +str(res["C_TOT_RMI_sum_"]* 1000))
print("Invest:    " +str(res["C_TOT_invAnn_sum_"]* 1000))
print("Total:     " +str(tot))

In [ ]:
demand = res["DH_Dem"]["Total"]
print(demand)

In [ ]:
Euro_per_H2 = tot/demand
print("EUR/kWh: " +str(Euro_per_H2))
print("EUR/kG:  " +str(Euro_per_H2*33.3))

### Show cost developmet of H2 in one chart

In [ ]:
y_ref_ = sc.params.y_ref_
years = sc.dims.Y
k__r_ = sc.params.k__r_
vals1 = {}
vals2 = {}
for year in years:
    res = util.get_price_per_year(self=sc, year=year)
    vals1[year] = ((res["C_TOT_op_sum_"] + res["C_TOT_RMI_sum_"] + res["C_TOT_invAnn_sum_"]) * 33300)/res["DH_Dem"]["Total"]
    vals2[year] = ((res["C_TOT_op_sum_"] + res["C_TOT_RMI_sum_"] + res["C_TOT_invAnn_sum_"]) * 1000)/res["DH_Dem"]["Total"]
#vals1
#vals2

In [ ]:
u1 = pd.DataFrame.from_dict(vals1,orient="index",columns=["Preis $\mathrm{H}_2$ pro kg"])
u2 = pd.DataFrame.from_dict(vals2,orient="index",columns=["Preis $\mathrm{H}_2$ pro kWh (rechts)"])
au = u1.plot.bar(title="Wasserstoff Kostenentwicklung", xlabel="Jahr", ylabel="Kosten [€/kg]")
u2.plot.bar(title="Wasserstoff Kostenentwicklung", xlabel="Jahr", ylabel="Kosten [€/kg]", secondary_y=True, ax=au, mark_right=False)
u

### Show energy flows per year

In [ ]:
draf.Scenario.get_all_collector_values = util.my_collector_values

In [ ]:
sc.year_sankey = 2027

In [ ]:
cs.plot.sankey_interact()

### Other

In [ ]:
sum(cs.scens.REF.res.P_PV_FI_TY[:,2027]+cs.scens.REF.res.P_PV_OC_TY[:,2027])

In [ ]:
sum(cs.scens.REF.res.P_Elc_TY[:,2040])

In [ ]:
sum(cs.scens.REF.res.dH_HDS_out_TYR[:,:,"NH3"][cs.scens.REF.res.dH_HDS_in_TYR[:,:,"NH3"] <= 0])